# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [8]:
#sc.stop()

In [1]:
import findspark
findspark.init()

import sys
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, functions as F, Window

sc = SparkContext(appName="SGA") 
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-10-25 11:37:41,216 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
df = se.read.csv("hdfs:/data/clickstream.csv", header=True, inferSchema=True, sep='\t')
df.show(20)

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584160|
|    562|       507|        page|    rabota|1695584166|
|    562|       507|       event|    rabota|1695584174|
|    562|       507|       event|    rabota|1695584181|
|    562|       507|       event|    rabota|1695584189|
|    562|       507|        page|      main|1695584194|
|    562|       507|       event|      main|1695584204|
|    562|       507|       event|      main|1695

# 1: Spark DF solution

In [3]:
# Remov and flag errors
df = df.withColumn("error_flag", F.when(F.lower(F.col("event_type")).contains("error"), 1).otherwise(0))
window_spec = Window.partitionBy("user_id", "session_id").orderBy("timestamp").rowsBetween(Window.unboundedPreceding, Window.currentRow)
df = df.withColumn("cumulative_error", F.sum("error_flag").over(window_spec))

# Filter to keep rows until the first error
filtered_df = df.filter(F.col("cumulative_error") == 0).drop("error_flag", "cumulative_error")

# Deduplicate within the session
window_session = Window.partitionBy("user_id", "session_id").orderBy("timestamp")
filtered_df = (
    filtered_df.withColumn("prev_page", F.lag("event_page").over(window_session))
               .filter((F.col("event_page") != F.col("prev_page")) | F.col("prev_page").isNull())
)

# Form routes and count
routes_df = (
    filtered_df.groupBy("user_id", "session_id")
               .agg(F.collect_list("event_page").alias("pages"))
               .withColumn("route", F.expr("array_join(pages, '-')"))
               .groupBy("route")
               .count()
               .orderBy(F.desc("count"))
               .limit(30)
)


In [4]:
import json

top_10_routes_df = routes_df.limit(10)

top_10_routes_dict = {row["route"]: row["count"] for row in top_10_routes_df.collect()}

with open("result.json", "w") as file:
    json.dump(top_10_routes_dict, file, indent=4)


In [6]:
!curl -F file=@result.json 51.250.123.136:80/MDS-LSML1/ilvira_khaidarova/w6/2

0.9999999999999999
Correct main answer!
Correct main-archive answer!
Correct main-rabota answer!
Correct main-internet answer!
Correct main-bonus answer!
Correct main-news answer!
Correct main-tariffs answer!
Correct main-online answer!
Correct main-vklad answer!
Correct main-rabota-archive answer!


# 2: Spark RDD solution

In [11]:
raw_rdd = sc.textFile("hdfs:/data/clickstream.csv")
header = raw_rdd.first()  
data_rdd = raw_rdd.filter(lambda row: row != header)  
data_rdd = data_rdd.map(lambda row: row.split("\t"))  

# (user_id, session_id, event_type, event_page, timestamp)
data_rdd = data_rdd.map(lambda x: (int(x[0]), int(x[1]), x[2], x[3], int(x[4])))

# Filter errors
session_errors = data_rdd.filter(lambda x: "error" in x[2].lower()).map(lambda x: ((x[0], x[1]), x[4]))

# Get timestamp for first error
first_error_ts = session_errors.reduceByKey(lambda a, b: min(a, b))

# Filter after the first error
data_rdd = data_rdd.map(lambda x: ((x[0], x[1]), x)) \
                   .leftOuterJoin(first_error_ts) \
                   .filter(lambda x: x[1][1] is None or x[1][0][4] < x[1][1]) \
                   .map(lambda x: x[1][0])

# Deduplicate & Sort
sorted_rdd = data_rdd.sortBy(lambda x: (x[0], x[1], x[4]))

def deduplicate_consecutive_pages(records):
    previous_page = None
    for record in records:
        if record[3] != previous_page:
            yield record[3]
        previous_page = record[3]

# Group by user_id, session_id
session_routes = sorted_rdd.map(lambda x: ((x[0], x[1]), x)).groupByKey() \
                           .mapValues(deduplicate_consecutive_pages) \
                           .mapValues(lambda pages: list(pages)) \
                           .filter(lambda x: len(x[1]) >= 1)  

# Aggregate and count
route_counts = session_routes.map(lambda x: ("-".join(x[1]), 1)).reduceByKey(lambda a, b: a + b)

top_10_routes = route_counts.takeOrdered(10, key=lambda x: -x[1])
for route, count in top_10_routes:
    print(f"{route}\t{count}")


main	8184
main-archive	1113
main-rabota	1047
main-internet	897
main-bonus	870
main-news	769
main-tariffs	677
main-online	587
main-vklad	518
main-rabota-archive	170


# 3: Spark SQL solution

In [7]:
# Remov and flag errors
df = df.withColumn("error_flag", F.when(F.lower(F.col("event_type")).contains("error"), 1).otherwise(0))
window_spec = Window.partitionBy("user_id", "session_id").orderBy("timestamp").rowsBetween(Window.unboundedPreceding, Window.currentRow)
df = df.withColumn("cumulative_error", F.sum("error_flag").over(window_spec))

# Filter to keep rows until the first error
filtered_df = df.filter(F.col("cumulative_error") == 0).drop("error_flag", "cumulative_error")

# Deduplicate within the session
window_session = Window.partitionBy("user_id", "session_id").orderBy("timestamp")
filtered_df = (
    filtered_df.withColumn("prev_page", F.lag("event_page").over(window_session))
               .filter((F.col("event_page") != F.col("prev_page")) | F.col("prev_page").isNull())
)

filtered_df.createOrReplaceTempView("filtered_clickstream")

In [10]:
# Use SQL to group by user and session, then aggregate and count 
routes_df = se.sql("""
    WITH session_routes AS (
        SELECT 
            user_id, 
            session_id, 
            COLLECT_LIST(event_page) AS pages
        FROM filtered_clickstream
        GROUP BY user_id, session_id
    ),
    route_counts AS (
        SELECT 
            ARRAY_JOIN(pages, '-') AS route, 
            COUNT(*) AS count
        FROM session_routes
        GROUP BY route
    )
    SELECT route, count
    FROM route_counts
    ORDER BY count DESC
    LIMIT 10
""")

routes_df.show(truncate=False)

+-------------------+-----+
|route              |count|
+-------------------+-----+
|main               |8185 |
|main-archive       |1112 |
|main-rabota        |1047 |
|main-internet      |897  |
|main-bonus         |870  |
|main-news          |769  |
|main-tariffs       |676  |
|main-online        |587  |
|main-vklad         |518  |
|main-rabota-archive|170  |
+-------------------+-----+

